# Get Area-Elevation Curve

In [1]:
import ee
ee.Initialize(project='tmospp')

In [2]:
# from rat.ee_utils.ee_aec_file_creator import aec_file_creator
import geopandas as gpd
from pathlib import Path
import hvplot.pandas
import pandas as pd
import holoviews as hv
import geoviews as gv
import numpy as np

hv.extension('bokeh')

### select the reservoir

In [3]:
start_date = '2022-01-01'
end_date = '2024-08-01'
RESERVOIR = '0810'
buffer_amt = 250 # meters. unlike other types of data, nadir altimetry data works better with a smaller buffer around the reservoir. Taking no buffer for now.
# possible_elevations_method = 'grand' #(60, 130)
# possible_elevations_method = [160, 950]
DATA_DIR = Path('/tiger1/pdas47/tmsosPP/data')
poly_deg = 2

In [4]:
# read the bounding box of the study area
val_pts = gpd.read_file(Path('/tiger1/pdas47/tmsosPP/data/validation-locations/100-validation-reservoirs-grand-pts.geojson'))
val_polys = gpd.read_file(Path('/tiger1/pdas47/tmsosPP/data/validation-locations/100-validation-reservoirs-grand-polys.geojson'))

selected_reservoirs = val_pts['tmsos_id'].tolist()
res_names = val_pts[['tmsos_id', 'name']].set_index('tmsos_id').to_dict()['name']

RESERVOIR_NAME = res_names[RESERVOIR]
print(f'{RESERVOIR}: {RESERVOIR_NAME}')

val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

nominal_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_SKM'].values[0]
nominal_area_poly = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_POLY'].values[0]
max_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MAX'].values[0]
max_area = np.nan if max_area == -99 else max_area
min_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MIN'].values[0]
min_area = 0 if min_area == -99 else min_area
area_rep = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_REP'].values[0]
dam_height = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DAM_HGT_M'].values[0])
elev_msl = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['ELEV_MASL'].values[0])
depth = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DEPTH_M'].values[0])
capacity = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['CAP_MCM'].values[0])

global_map = (
    val_res_pt.hvplot(
        geo=True, tiles='OSM'
    ) * val_res_pt[val_res_pt['tmsos_id'] == RESERVOIR].hvplot(
        geo=True, color='red', size=100, 
    )
).opts(
    title=f"Locations of validation reservoirs. {RESERVOIR_NAME}, highlighted in red"
)

global_map

0810: Noi, Th


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]
   .Points.II :Points   [Longitude,Latitude]

## Storage Calculation

In [5]:
# what is the reported capacity?
capacity_hv = hv.HLine(capacity).opts(color='red', ylim=(0, capacity + capacity*0.1), ylabel='capacity (Mil. m3)')
capacity_hv

:HLine   [x,y]

In [6]:
aec_dir = Path('/tiger1/pdas47/tmsosPP/data/aec/aev_2deg_poly_constraints_centerline_and_dam_buffer')

In [7]:
val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

aec_fp = aec_dir / f'{RESERVOIR}.csv'
aec_df = pd.read_csv(aec_fp, comment='#')

from scipy.integrate import cumulative_trapezoid
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.cumulative_trapezoid.html#scipy.integrate.cumulative_trapezoid
## Cumulatively integrate y(x) using the composite trapezoidal rule.

elevation_normalized = (aec_df['Elevation'] - aec_df['Elevation'].min())

# cumulative_trapezoid takes two parameters.
# y = y-axis locations of points. these values will be integrated. https://en.wikipedia.org/wiki/File:Composite_trapezoidal_rule_illustration.png. Normalized Elevation.
# x = x-axis locations of points, where each y value is sampled. Area.
storage = cumulative_trapezoid(elevation_normalized, aec_df['CumArea'] * 1e6)
storage = np.insert(storage, 0, 0)

aec_df['Storage'] = storage
aec_df['Storage (mil. m3)'] = storage * 1e-6
aec_df

,CumArea,Elevation,Storage,Storage (mil. m3),Elevation_Observed
0,0.00,114.062737,0.000000e+00,0.000000,NaN
1,0.25,114.092238,3.687715e+03,0.003688,NaN
2,0.50,114.121739,1.475077e+04,0.014751,NaN
3,0.75,114.151239,3.318897e+04,0.033189,NaN
4,1.00,114.180739,5.900213e+04,0.059002,NaN
...,...,...,...,...,...
1459,361.00,155.883742,7.595622e+09,7595.622389,NaN
1460,361.25,155.912163,7.606081e+09,7606.081193,NaN
1461,361.50,155.940584,7.616547e+09,7616.547102,NaN
1462,361.75,155.969003,7.627020e+09,7627.020117,NaN


In [8]:
aec_df.hvplot.scatter(
    x='CumArea', y='Elevation', 
    # by='obs_or_extrapolated'
).opts(
    height=400, width=500,
    title=f'{RESERVOIR}: {RESERVOIR_NAME}\nAEC',
    xlabel='Area (km2)', ylabel='Elevation (m a.s.l)'
)

:Scatter   [CumArea]   (Elevation)

In [9]:
aec_df.hvplot(x='Elevation', y='CumArea').opts(height=400, width=500, title=f'{RESERVOIR}: {RESERVOIR_NAME}  [A-E]') + aec_df.hvplot(x='Elevation', y='Storage (mil. m3)', title=f'{RESERVOIR}: {RESERVOIR_NAME}  [S-E]').opts(height=400, width=500) * capacity_hv

:Layout
   .Curve.I   :Curve   [Elevation]   (CumArea)
   .Overlay.I :Overlay
      .Curve.I :Curve   [Elevation]   (Storage (mil. m3))
      .HLine.I :HLine   [x,y]

In [10]:
# ## save aec in `srtm_extrapolated_storage`
# srtm_extrapolated_dir = Path('/tiger1/pdas47/tmsosPP/data/aec/srtm_extrapolated_storage/')
# # aec_fp = srtm_extrapolated_dir / f'{RESERVOIR}_poly_{poly_deg}_storage.csv'
# # aec_df.to_csv(aec_fp, index=False)

### TMS-OS

In [11]:
import xarray as xr
import hvplot.xarray
import numpy as np

alg_type = 'tmsos'
elevation_dir = Path('/tiger1/pdas47/tmsosPP/data/tmsos/')
elevation_fp = elevation_dir / f'{RESERVOIR}.csv'

data = {} # data to be used in the xarray
if alg_type == 'tmsos':
    tmsos_df = pd.read_csv(elevation_fp, parse_dates=['date'])
    data['area'] = tmsos_df['area']
    data['date'] = tmsos_df['date'] # add area and date from the tmsos data

reservoir_dynamics = pd.DataFrame(data).set_index('date').to_xarray() # convert to xarray

# compute elevation values using aec
if alg_type == 'tmsos':
    reservoir_dynamics['area'].attrs['alg_type'] = 'tmsos'
    reservoir_dynamics['area'].attrs['obs_imp'] = 'obs'
    reservoir_dynamics['area'].attrs['unit'] = 'km^2'

    elevation = np.interp(reservoir_dynamics['area'], aec_df['CumArea'], aec_df['Elevation'])
    elevation_da = xr.DataArray(data=elevation, coords=reservoir_dynamics.coords)

    storage = np.interp(reservoir_dynamics['area'], aec_df['CumArea'], aec_df['Storage'])
    storage_da = xr.DataArray(data=storage, coords=reservoir_dynamics.coords)

    reservoir_dynamics = reservoir_dynamics.assign(elevation = elevation_da)
    reservoir_dynamics = reservoir_dynamics.assign(storage = storage_da)

# compute storage change
if 'storage_change' not in list(reservoir_dynamics.variables):
    avg_A = (reservoir_dynamics['area'].isel(date=slice(0, -1)) + reservoir_dynamics['area'].isel(date=slice(1, None)))/2
    del_h = reservoir_dynamics['elevation'].diff(dim='date')
    del_s = xr.DataArray(0.5 * avg_A * del_h * 1e6, name='storage_change')
    reservoir_dynamics = reservoir_dynamics.assign(storage_change=del_s)

reservoir_dynamics

<xarray.Dataset> Size: 22kB
Dimensions:         (date: 562)
Coordinates:
  * date            (date) datetime64[ns] 4kB 2019-01-16 ... 2024-08-27
Data variables:
    area            (date) float64 4kB 189.3 189.3 189.6 ... 187.3 187.3 187.3
    elevation       (date) float64 4kB 136.2 136.2 136.2 ... 136.0 136.0 136.0
    storage         (date) float64 4kB 2.102e+09 2.102e+09 ... 2.057e+09
    storage_change  (date) float64 4kB nan 0.0 2.688e+06 ... 0.0 0.0 nan

In [12]:
print(capacity)
(capacity_hv * (reservoir_dynamics*1e-6).hvplot.scatter(x='date', y='storage').opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME}. Storage (mil. m3)', ylabel='Storage', xlabel='Date'
))

1966.0


:Overlay
   .HLine.I   :HLine   [x,y]
   .Scatter.I :Scatter   [date]   (storage)

In [13]:
print(capacity)
(capacity_hv * (reservoir_dynamics*1e-6).hvplot.scatter(x='date', y='storage').opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME}. Storage (mil. m3)', ylabel='Storage', xlabel='Date'
))

1966.0


:Overlay
   .HLine.I   :HLine   [x,y]
   .Scatter.I :Scatter   [date]   (storage)

In [14]:
reservoir_dynamics.hvplot.scatter(x='date', y='area').opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME}. Area (km2)'
) 
# + reservoir_dynamics.hvplot.scatter(x='date', y='elevation').opts(
#     title=f'{RESERVOIR}: {RESERVOIR_NAME}. Elevation (m)'
# ) + reservoir_dynamics.hvplot.scatter(
#     x='elevation', y='area', by='date'
# )).cols(1)

:Scatter   [date]   (area)

In [15]:
print(capacity)
reservoir_dynamics.hvplot.scatter(x='date', y='storage_change').opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME}. Storage Change (million m^3)'
)

1966.0


:Scatter   [date]   (storage_change)

In [16]:
satellite_reservoir_dynamics = reservoir_dynamics

In [17]:
VERSION = '0.1'
ALG = 'tmsos_aev_2deg_poly_constraints_centerline_and_dam_buffer'

csv_save_fp = DATA_DIR / 'storage' / ALG / f'v{VERSION}' / f'{RESERVOIR}_{RESERVOIR_NAME.split(",")[0].replace(" ", "_")}_storage.csv'
csv_save_fp.parent.mkdir(parents=True, exist_ok=True)
if csv_save_fp.exists():
    input(f"Press enter to save.. file exists at {csv_save_fp}")
    satellite_reservoir_dynamics.to_pandas().to_csv(csv_save_fp)
else:
    satellite_reservoir_dynamics.to_pandas().to_csv(csv_save_fp)

In [18]:
nc_save_fp = csv_save_fp.with_suffix('.nc')
# save as netcdf
## add attributes

satellite_reservoir_dynamics.attrs['version'] = VERSION
satellite_reservoir_dynamics.to_netcdf(nc_save_fp)

# [STORAGE CHANGE] Read in-situ data and calculate storage change

## Select the reservoir

In [17]:
# # RESERVOIR = '0505'
# ALG_VERSION = 'v0.1.1' # remove temporal resampling

# RESULTS_DIR = Path(f'/tiger1/pdas47/tmsosPP/results/')
# DATA_DIR = Path(f'/tiger1/pdas47/tmsosPP/data')

In [18]:
# # read the bounding box of the study area
# val_pts = gpd.read_file(Path('/tiger1/pdas47/tmsosPP/data/validation-locations/100-validation-reservoirs-grand-pts.geojson'))
# val_polys = gpd.read_file(Path('/tiger1/pdas47/tmsosPP/data/validation-locations/100-validation-reservoirs-grand-polys.geojson'))

# RESERVOIR_NAME = res_names[RESERVOIR]

# val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
# val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

# global_map = (
#     val_res_pt.hvplot(
#         geo=True, tiles='OSM'
#     ) * val_res_pt[val_res_pt['tmsos_id'] == RESERVOIR].hvplot(
#         geo=True, color='red', size=100, 
#     )
# ).opts(
#     title=f"Locations of validation reservoirs. {RESERVOIR_NAME}, highlighted in red"
# )

# global_map

In [19]:
# (val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR].hvplot(
#     geo=True, tiles='OSM', shared_axes=False
# )).opts(title=f"{RESERVOIR_NAME}")

## Read insitu and satellite data

In [20]:
# import numpy as np

# VERSION = '0.1'
# deltares_insitu_dir = Path('/tiger1/pdas47/tmsosPP/data/insitu/deltares/')
# rid_insitu_dir = Path('/tiger1/pdas47/tmsosPP/data/insitu/rid')
# resops_insitu_dir = Path('/tiger1/pdas47/tmsosPP/data/insitu/resopsus')
# area_column = 'tmsos area [km2]'
# area_dir = Path('/tiger1/pdas47/tmsosPP/data/area/tmsos/')



# insitu_dfs = []
# sat_dfs = []
# test_dfs = []

# perf_dfs = []

# for reservoir in selected_reservoirs:
#     insitu_df = get_insitu_df(reservoir)

#     insitu_df['tmsos_id'] = reservoir
#     insitu_df.set_index(['tmsos_id', 'date'], inplace=True)
#     insitu_dfs.append(insitu_df)
    
#     sat_fn = Path(f'{area_dir}/v{VERSION}/{reservoir}.csv')
#     sat_df = pd.read_csv(sat_fn, parse_dates=['time'], dtype={'tmsos_id': str})
#     sat_df['date'] = pd.to_datetime(sat_df['time'].dt.date)
#     sat_df = sat_df.drop(['time'], axis=1)
#     sat_df.set_index(['tmsos_id', 'date'], inplace=True)
#     sat_dfs.append(sat_df)

# insitu_df = pd.concat(insitu_dfs)
# sat_df = pd.concat(sat_dfs)

In [21]:
# import xarray as xr

# srtm_extrapolated_dir = Path('/tiger1/pdas47/tmsosPP/data/aec/srtm_extrapolated')
# # poly_deg = 3
# aec_fp = srtm_extrapolated_dir / f'{RESERVOIR}_poly_{poly_deg}.csv'
# aec_df = pd.read_csv(aec_fp)
# alg_type = 'insitu'

# insitu_df_res = insitu_df.reset_index()
# insitu_df_res = insitu_df_res[insitu_df_res['tmsos_id'] == RESERVOIR]
# insitu_ds_res = insitu_df_res.set_index(['date']).to_xarray()

# reservoir_dynamics = insitu_df_res.rename({
#     'observed area [km2]': 'area',
#     'observed wse [m]': 'wse',
#     'observed storage [Mm3]': 'storage'
# }, axis=1).drop_duplicates('date').set_index('date').to_xarray()

# # compute values
# if alg_type == 'insitu':
#     reservoir_dynamics['area'].attrs['alg_type'] = 'insitu'
#     reservoir_dynamics['area'].attrs['obs_imp'] = 'obs'
#     reservoir_dynamics['area'].attrs['unit'] = 'km^2'

#     elevation = np.interp(reservoir_dynamics['area'], aec_df['CumArea'], aec_df['Elevation'])
#     elevation_da = xr.DataArray(data=elevation, coords=reservoir_dynamics.coords)
#     reservoir_dynamics = reservoir_dynamics.assign(elevation = elevation_da)
# # similarly, add other custom functions for other alg_types

# if 'storage_change' not in list(reservoir_dynamics.variables):
#     avg_A = (reservoir_dynamics['area'].isel(date=slice(0, -1)) + reservoir_dynamics['area'].isel(date=slice(1, None)))/2
#     del_h = reservoir_dynamics['elevation'].diff(dim='date')
#     del_s = xr.DataArray(0.5 * avg_A * del_h, name='storage_change')
#     reservoir_dynamics = reservoir_dynamics.assign(storage_change=del_s)

# reservoir_dynamics

In [22]:
# insitu_df_res.hvplot(x='date', y='observed area [km2]', kind='scatter')

In [23]:
# import hvplot.xarray
# import holoviews as hv
# hv.extension('bokeh')

# reservoir_dynamics.hvplot(
#     x='date', y='storage_change', label='insitu', color='red', kind='scatter'
# ).opts(
#     title=f'{RESERVOIR}: {RESERVOIR_NAME}. comparing ∆S between insitu and satellite'
# ) * satellite_reservoir_dynamics.hvplot(
#     x='date', y='storage_change', label='satellite (tmsos)', color='blue', alpha=0.6, kind='scatter'
# ).opts(
#     xlim=tuple(pd.to_datetime(('2019-01-01', '2024-07-01')).values), 
#     ylim=(-200, 200),
#     ylabel='∆S (Mm^3)',
# )

In [24]:

# alg_type = 'tmsos'
# elevation_dir = Path('/tiger1/pdas47/tmsosPP/data/tmsos/')
# elevation_fp = elevation_dir / f'{RESERVOIR}.csv'

# data = {}
# if alg_type == 'tmsos':
#     tmsos_df = pd.read_csv(elevation_fp, parse_dates=['date'])
#     data['area'] = tmsos_df['area']
#     data['date'] = tmsos_df['date']

# reservoir_dynamics = pd.DataFrame(data).set_index('date').to_xarray()

# # compute values
# if alg_type == 'tmsos':
#     reservoir_dynamics['area'].attrs['alg_type'] = 'tmsos'
#     reservoir_dynamics['area'].attrs['obs_imp'] = 'obs'
#     reservoir_dynamics['area'].attrs['unit'] = 'km^2'

#     elevation = np.interp(reservoir_dynamics['area'], aec['CumArea'], aec['Elevation'])
#     elevation_da = xr.DataArray(data=elevation, coords=reservoir_dynamics.coords)
#     elevation_da.attrs['alg_type'] = 'tmsos'
#     elevation_da.attrs['obs_imp'] = 'imp'
#     elevation_da.attrs['unit'] = 'm'
    
#     reservoir_dynamics = reservoir_dynamics.assign(elevation = elevation_da)

# # similarly, add other custom functions for other alg_types
# if 'storage_change' not in list(reservoir_dynamics.variables):
#     avg_A = (reservoir_dynamics['area'] + reservoir_dynamics['area'].shift({'date':1}))/2
#     del_h = reservoir_dynamics['elevation'] - reservoir_dynamics['elevation'].shift({'date':1})
    
#     del_s = xr.DataArray(avg_A * del_h * 1e6, name='storage_change')
#     del_s = xr.DataArray(del_s, name='storage_change')
#     del_s.attrs['alg_type'] = 'tmsos'
#     del_s.attrs['obs_imp'] = 'calc'
#     del_s.attrs['unit'] = 'm^3'
    
#     reservoir_dynamics = reservoir_dynamics.assign(storage_change=del_s)

# reservoir_dynamics

In [25]:
# reservoir_dynamics.storage_change.hvplot(kind='scatter', color='storage_change', cmap='RdYlGn')

## Save 

In [26]:
# VERSION = '0.1'
# ALG = 'insitu'

# csv_save_fp = DATA_DIR / 'storage_change' / ALG / f'v{VERSION}' / f'{RESERVOIR}_{RESERVOIR_NAME.split(",")[0].replace(" ", "_")}_dels.csv'
# csv_save_fp.parent.mkdir(parents=True, exist_ok=True)
# reservoir_dynamics.to_pandas().to_csv(csv_save_fp)

In [27]:
# nc_save_fp = csv_save_fp.with_suffix('.nc')
# # save as netcdf
# ## add attributes

# reservoir_dynamics.attrs['version'] = VERSION
# reservoir_dynamics.to_netcdf(nc_save_fp)